**The idea for this project is recommend where should I open a restaurant based on where most of them are located**

In [7]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans

**Setup user credencials**

In [1]:
CLIENT_ID = 'WSSRJ0MGT4511UDY3ZC2K2GX5UZ45QHZQLUVCY0YR450UX1I' # your Foursquare ID
CLIENT_SECRET = 'SIBXOOMEDLNAHZW4T1UXT0IVHLVL4QJRX0C03VPYFPYH0GZL' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT =  100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WSSRJ0MGT4511UDY3ZC2K2GX5UZ45QHZQLUVCY0YR450UX1I
CLIENT_SECRET:SIBXOOMEDLNAHZW4T1UXT0IVHLVL4QJRX0C03VPYFPYH0GZL


<b>I locate the city where I will open the restaurant with latitude and longitude
the city's name is Barranquilla </b>

In [28]:
latitude = 10.96854 
longitude = -74.78132
radius = 10000

**Receiving information from Foursquare**

In [29]:


# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url 

'https://api.foursquare.com/v2/venues/explore?&client_id=WSSRJ0MGT4511UDY3ZC2K2GX5UZ45QHZQLUVCY0YR450UX1I&client_secret=SIBXOOMEDLNAHZW4T1UXT0IVHLVL4QJRX0C03VPYFPYH0GZL&v=20180605&ll=10.96854,-74.78132&radius=10000&limit=100'

In [30]:
results = requests.get(url).json()


**Creating Dataframe and filtering for Restaurant**

In [31]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [36]:
venues = results['response']['groups'][0]['items']

places = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
places =places.loc[:, filtered_columns]

# filter the category for each row
places['venue.categories'] = places.apply(get_category_type, axis=1)

# clean columns
places.columns = [col.split(".")[-1] for col in places.columns]

places.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,El Arriero,Latin American Restaurant,10.982056,-74.781795
1,Parrilla La Libanesa,Steakhouse,10.983910,-74.784116
2,Zamba Casino Restaurante Bar,Casino,10.990230,-74.788505
3,Museo del Caribe,Museum,10.986431,-74.778316
4,Teatro Rex Restaurante,Restaurant,10.984908,-74.779802


In [37]:
places.shape

(100, 4)

In [46]:
restaurant = places[places.categories.str.contains('Restaurant')]
restaurant.head()

,name,categories,lat,lng
0,El Arriero,Latin American Restaurant,10.982056,-74.781795
4,Teatro Rex Restaurante,Restaurant,10.984908,-74.779802
5,Picasso Ostrería,Seafood Restaurant,10.990889,-74.795436
7,Noa Sushi & Robata,Japanese Restaurant,11.003397,-74.808010
9,Arepas Y Arepas,Fast Food Restaurant,10.997946,-74.805695


**The map is creating with the restaurants**

In [52]:
import folium

# create map of New York using latitude and longitude values
Barranquillamap = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, name, categories in zip(restaurant['lat'], restaurant['lng'], restaurant['name'], restaurant['categories']):
    label = '{}, {}'.format(name, categories)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(Barranquillamap)  
    
Barranquillamap

**The map shows that most of the restaurants are located in the north zone**
